# Regression
Next you'll learn to create Linear Regression models. You'll also find out how to augment your data by engineering new predictors as well as a robust approach to selecting only the most relevant predictors.

## Encoding flight origin
The org column in the flights data is a categorical variable giving the airport from which a flight departs.

- ORD — O'Hare International Airport (Chicago)
- SFO — San Francisco International Airport
- JFK — John F Kennedy International Airport (New York)
- LGA — La Guardia Airport (New York)
- SMF — Sacramento
- SJC — San Jose
- TUS — Tucson International Airport
- OGG — Kahului (Hawaii)

Obviously this is only a small subset of airports. Nevertheless, since this is a categorical variable, it needs to be one-hot encoded before it can be used in a regression model.

The data are in a variable called flights. You have already used a string indexer to create a column of indexed values corresponding to the strings in org.

In [2]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import round, col


# File Path
file_path = ".../data/datacamp/"

# Load in data
flights = spark.read.parquet(file_path + "flights_ml") \
    .drop('flight') \
    .filter('delay IS NOT NULL') \
    .dropna() \
    .withColumn('km', round(col("mile") * 1.60934, 0)) \
    .drop('mile')

# Repeat the process for the other categorical feature
flights = StringIndexer(inputCol='org', outputCol='org_idx') \
    .fit(flights) \
    .transform(flights)

In [3]:
# Import the one hot encoder class
from pyspark.ml.feature import OneHotEncoderEstimator

# Create an instance of the one hot encoder
onehot = OneHotEncoderEstimator(inputCols=['org_idx'], outputCols=['org_dummy'])

# Apply the one hot encoder to the flights data
onehot = onehot.fit(flights)
flights_onehot = onehot.transform(flights)

# Check the results
flights_onehot.select('org', 'org_idx', 'org_dummy').distinct().sort('org_idx').show()

+---+-------+-------------+
|org|org_idx|    org_dummy|
+---+-------+-------------+
|ORD|    0.0|(7,[0],[1.0])|
|SFO|    1.0|(7,[1],[1.0])|
|JFK|    2.0|(7,[2],[1.0])|
|LGA|    3.0|(7,[3],[1.0])|
|SMF|    4.0|(7,[4],[1.0])|
|SJC|    5.0|(7,[5],[1.0])|
|TUS|    6.0|(7,[6],[1.0])|
|OGG|    7.0|    (7,[],[])|
+---+-------+-------------+



## Flight duration model: Just distance
In this exercise you'll build a regression model to predict flight duration (the duration column).

For the moment you'll keep the model simple, including only the distance of the flight (the km column) as a predictor.

The data are in flights. The first few records are displayed in the terminal. These data have also been split into training and testing sets and are available as flights_train and flights_test.

In [4]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Create an assembler object and split into train and test data
flights_train, flights_test = VectorAssembler(inputCols=['km'], outputCol='features') \
    .transform(flights) \
    .select('features', 'duration') \
    .randomSplit([0.8, 0.2], seed=17) 

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
predictions = regression.transform(flights_test)
predictions.select('duration', 'prediction').show(5, False)

# Calculate the RMSE
RegressionEvaluator(labelCol='duration').evaluate(predictions)

+--------+-----------------+
|duration|prediction       |
+--------+-----------------+
|43      |53.52566757005931|
|45      |53.67698095411485|
|41      |54.58486125844807|
|43      |54.58486125844807|
|43      |54.58486125844807|
+--------+-----------------+
only showing top 5 rows



16.95242404982233

## Interpreting the coefficients
The linear regression model for flight duration as a function of distance takes the form

duration=α+β×distance where

- α — intercept (component of duration which does not depend on distance) and
- β — coefficient (rate at which duration increases as a function of distance; also called the slope).

By looking at the coefficients of your model you will be able to infer

how much of the average flight duration is actually spent on the ground and
what the average speed is during a flight.

In [5]:
# Intercept (average minutes on ground)
inter = regression.intercept
print(inter)

# Coefficients
coefs = regression.coefficients
print(coefs)

# Average minutes per km
minutes_per_km = regression.coefficients[0]
print(minutes_per_km)

# Average speed in km per hour
avg_speed = 60 / minutes_per_km
print(avg_speed)

44.14423775861602
[0.0756566920277685]
0.0756566920277685
793.056085216864


## Flight duration model: Adding origin airport
Some airports are busier than others. Some airports are bigger than others too. Flights departing from large or busy airports are likely to spend more time taxiing or waiting for their takeoff slot. So it stands to reason that the duration of a flight might depend not only on the distance being covered but also the airport from which the flight departs.

You are going to make the regression model a little more sophisticated by including the departure airport as a predictor.

These data have been split into training and testing sets and are available as flights_train and flights_test. The origin airport, stored in the org column, has been indexed into org_idx, which in turn has been one-hot encoded into org_dummy. The first few records are displayed in the terminal.

In [6]:
# Create an assembler object and split into train and test data
flights_train, flights_test = VectorAssembler(inputCols=['km', 'org_dummy'], outputCol='features') \
    .transform(flights_onehot) \
    .select('features', 'duration') \
    .randomSplit([0.8, 0.2], seed=17) 

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

11.047073489399853

## Interpreting coefficients
Remember that origin airport, org, has eight possible values (ORD, SFO, JFK, LGA, SMF, SJC, TUS and OGG) which have been one-hot encoded to seven dummy variables in org_dummy.

The values for km and org_dummy have been assembled into features, which has eight columns with sparse representation. Column indices in features are as follows:

- 0 — km
- 1 — ORD
- 2 — SFO
- 3 — JFK
- 4 — LGA
- 5 — SMF
- 6 — SJC and
- 7 — TUS.

Note that OGG does not appear in this list because it is the reference level for the origin airport category.

In this exercise you'll be using the intercept and coefficients attributes to interpret the model.

In [7]:
# Average speed in km per hour
avg_speed_hour = 60 / regression.coefficients[0]
print(avg_speed_hour)

# Average minutes on ground at OGG
inter = regression.intercept
print(inter)

# Average minutes on ground at JFK
avg_ground_jfk = inter + regression.coefficients[3]
print(avg_ground_jfk)

# Average minutes on ground at LGA
avg_ground_lga = inter + regression.coefficients[4]
print(avg_ground_lga)

807.9141639544247
15.897612012321108
68.55756045369932
62.75551502718034


You're going to spend over an hour on the ground at JFK or LGA but only around 15 minutes at OGG.

## Bucketing departure time
Time of day data are a challenge with regression models. They are also a great candidate for bucketing.

In this lesson you will convert the flight departure times from numeric values between 0 (corresponding to 00:00) and 24 (corresponding to 24:00) to binned values. You'll then take those binned values and one-hot encode them.

In [8]:
from pyspark.ml.feature import Bucketizer, OneHotEncoderEstimator

# Create buckets at 3 hour intervals through the day
buckets = Bucketizer(splits=[0, 3, 6, 9, 12, 15, 18, 21, 24], inputCol='depart', outputCol='depart_bucket')

# Bucket the departure times
bucketed = buckets.transform(flights)
bucketed.select('depart', 'depart_bucket').show(5)

# Create a one-hot encoder
onehot = OneHotEncoderEstimator(inputCols=['depart_bucket', 'org_idx'], outputCols=['depart_dummy', 'org_dummy'])

# One-hot encode the bucketed departure times
flights_onehot = onehot.fit(bucketed).transform(bucketed)
flights_onehot.select('depart', 'depart_bucket', 'depart_dummy').show(5)

+------+-------------+
|depart|depart_bucket|
+------+-------------+
| 12.75|          4.0|
|  9.07|          3.0|
| 20.75|          6.0|
| 13.17|          4.0|
| 16.67|          5.0|
+------+-------------+
only showing top 5 rows

+------+-------------+-------------+
|depart|depart_bucket| depart_dummy|
+------+-------------+-------------+
| 12.75|          4.0|(7,[4],[1.0])|
|  9.07|          3.0|(7,[3],[1.0])|
| 20.75|          6.0|(7,[6],[1.0])|
| 13.17|          4.0|(7,[4],[1.0])|
| 16.67|          5.0|(7,[5],[1.0])|
+------+-------------+-------------+
only showing top 5 rows



## Flight duration model: Adding departure time
In the previous exercise the departure time was bucketed and converted to dummy variables. Now you're going to include those dummy variables in a regression model for flight duration.

The data are in flights. The km, org_dummy and depart_dummy columns have been assembled into features, where km is index 0, org_dummy runs from index 1 to 7 and depart_dummy from index 8 to 14.

In [9]:
# Create an assembler object and split into train and test data
flights_train, flights_test = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy' ], outputCol='features') \
    .transform(flights_onehot) \
    .select('features', 'duration') \
    .randomSplit([0.8, 0.2], seed=17) 

# Create a regression object and train on training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Create predictions for the testing data
predictions = regression.transform(flights_test)

# Find the RMSE on testing data
RegressionEvaluator(labelCol='duration').evaluate(predictions)

# Average minutes on ground at OGG for flights departing between 21:00 and 24:00
avg_eve_ogg = regression.intercept
print(avg_eve_ogg)

# Average minutes on ground at OGG for flights departing between 00:00 and 03:00
avg_night_ogg = regression.intercept + regression.coefficients[8]
print(avg_night_ogg)

# Average minutes on ground at JFK for flights departing between 00:00 and 03:00
avg_night_jfk = regression.intercept + regression.coefficients[8] + regression.coefficients[3]
print(avg_night_jfk)

9.93189322886289
-4.361844845037364
47.673598352887055


## Flight duration model: More features!
Let's add more features to our model. This will not necessarily result in a better model. Adding some features might improve the model. Adding other features might make it worse.

More features will always make the model more complicated and difficult to interpret.

These are the features you'll include in the next model:

- km
- org (origin airport, one-hot encoded, 8 levels)
- depart (departure time, binned in 3 hour intervals, one-hot encoded, 8 levels)
- dow (departure day of week, one-hot encoded, 7 levels) and
- mon (departure month, one-hot encoded, 12 levels).

These have been assembled into the features column, which is a sparse representation of 32 columns (remember one-hot encoding produces a number of columns which is one fewer than the number of levels).

In [10]:
# One-hot encode the bucketed departure times
flights_onehot = OneHotEncoderEstimator(inputCols=['dow', 'mon'], outputCols=['dow_dummy', 'mon_dummy']) \
                                        .fit(flights_onehot) \
                                        .transform(flights_onehot)

# Create an assembler object and split into train and test data
flights_train, flights_test = VectorAssembler(inputCols=['km', 'org_dummy', 'depart_dummy',  'dow_dummy', 'mon_dummy'], outputCol='features') \
    .transform(flights_onehot) \
    .select('features', 'duration') \
    .randomSplit([0.8, 0.2], seed=17) 

# Fit linear regression model to training data
regression = LinearRegression(labelCol='duration').fit(flights_train)

# Make predictions on testing data
predictions = regression.transform(flights_test)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(predictions)
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

The test RMSE is 10.582128563167197
[0.0743319472996046,27.54479958179488,20.35370169602556,52.17147858248114,46.298397809597404,15.432197814194582,17.655739990791822,17.86805500841664,-14.549346560840478,0.34176820070101593,4.1996550138547795,7.180131332582136,4.843764412827084,8.971098158037346,9.080797831310239,-0.046114197943885014,-0.1047424547426897,-0.09596441499872434,-0.024955513458476228,-0.030252367322806898,-0.0678609436374659,-1.72776455934809,-1.9400391171232136,-2.006674257576216,-3.344464825577119,-3.9896057778330403,-3.8683667664500354,-3.9318563574558754,-3.94124456340602,-3.7079215093143763,-2.510194447489106,-0.4020077907788954]


With all those non-zero coefficients the model is a little hard to interpret!

## Flight duration model: Regularisation!
In the previous exercise you added more predictors to the flight duration model. The model performed well on testing data, but with so many coefficients it was difficult to interpret.

In this exercise you'll use Lasso regression (regularized with a L1 penalty) to create a more parsimonious model. Many of the coefficients in the resulting model will be set to zero. This means that only a subset of the predictors actually contribute to the model. Despite the simpler model, it still produces a good RMSE on the testing data.

You'll use a specific value for the regularization strength. Later you'll learn how to find the best value using cross validation.

In [11]:
# Fit Lasso model (α = 1) to training data
regression = LinearRegression(labelCol='duration', regParam=1, elasticNetParam=1).fit(flights_train)

# Calculate the RMSE on testing data
rmse = RegressionEvaluator(labelCol='duration').evaluate(regression.transform(flights_test))
print("The test RMSE is", rmse)

# Look at the model coefficients
coeffs = regression.coefficients
print(coeffs)

# Number of zero coefficients
zero_coeff = sum([beta == 0 for beta in regression.coefficients])
print("Number of coefficients equal to 0:", zero_coeff)

The test RMSE is 11.533641176887867
[0.07349681473143059,5.63897869797241,0.0,29.24849547122284,22.242571518711596,-2.0582524281427137,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0342407810591896,1.2772394293700793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
Number of coefficients equal to 0: 25


Regularisation produced a far simpler model with similar test performance.